In [12]:
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Загрузка данных из файла
data = pd.read_excel('questions.xlsx')

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased', force_download=True)
model = model.to(torch.device('cpu'))

# Функция для обработки вопроса и получения ответа
def get_answer(question, context):
    # Токенизация вопроса и контекста
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Получение ответа с помощью модели BERT
    start_scores, end_scores = model(input_ids, attention_mask=attention_mask)

    # Индексы начала и конца ответа
    start_index = torch.argmax(start_scores[0])
    end_index = torch.argmax(end_scores[0])


    # Получение ответа из токенизированного текста
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index+1]))

    return answer

# Выбор случайного вопроса
random_question = data.sample(1)

# Вывод вопроса
print('Вопрос:', random_question['question'].values[0])

# Ввод ответа респондента
response = input('Ответ: ')

# Обработка ответа и получение предсказанного ответа
predicted_answer = get_answer(random_question['question'].values[0], response)

# Сравнение предсказанного ответа с ожидаемым ответом
expected_answer = random_question['answer'].values[0]
if predicted_answer.lower() == expected_answer.lower():
    print('Ответ верный')
else:
    print('Ответ неверный')


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Вопрос: Что такое диапазон


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str